## TheMealDB API — Endpoint + Parameter Setup

https://www.themealdb.com/api.php

We are going to use TheMealDB API, and we will not manually attach `?something=` at the end of URLs.  
Instead, we pass parameters through the `params={}` argument in `requests.get()`.

### Section 1: Endpoints we will use:

| Task | Endpoint URL | Required Parameter | Meaning |
|---|---|---|---|
| Search meals by name text | `https://www.themealdb.com/api/json/v1/1/search.php` | `s` | string to search inside meal names |
| Filter meals by category | `https://www.themealdb.com/api/json/v1/1/filter.php` | `c` | exact category name (Example: `Dessert`) |
| Lookup full details by ID | `https://www.themealdb.com/api/json/v1/1/lookup.php` | `i` | the exact `idMeal` value |

> **Important:** `q=` does not work here — TheMealDB does not use `q`.  
> These 3 parameter keys — `s`, `c`, `i` — are the official ones.




### Why we do it this way
APIs require exact field names.  
LLMs may guess parameter names — but APIs do not guess.  
So we always check parameter directly in documentation.


**Exampple Prompt to LLM:**

I want to find meals that are desserts. I know TheMealDB has multiple endpoints and each endpoint uses a different parameter name.

The endpoints and required parameters are:

1. https://www.themealdb.com/api/json/v1/1/search.php: "s"
2. https://www.themealdb.com/api/json/v1/1/filter.php: "c"
3. https://www.themealdb.com/api/json/v1/1/lookup.php: "i"


Which endpoint and parameter should I call to retrieve a list of Dessert meals?

Do NOT give me the meals yet.  
Just tell me which endpoint + parameter to call, in the format:

endpoint: <…>
params: { "<key>": "<value>" }

I will run the API call myself and paste the result back to you.


**Possible LLM answer:**

endpoint: /filter.php  
params: { "c": "dessert" }


In [1]:
import requests

url = "https://www.themealdb.com/api/json/v1/1/filter.php"   # endpoint
params = { "c": "dessert" }                                  # search term

r = requests.get(url, params=params)
dessert = r.json()

dessert



{'meals': [{'strMeal': 'Apam balik',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/adxcbq1619787919.jpg',
   'idMeal': '53049'},
  {'strMeal': 'Apple & Blackberry Crumble',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/xvsurr1511719182.jpg',
   'idMeal': '52893'},
  {'strMeal': 'Apple Frangipan Tart',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/wxywrq1468235067.jpg',
   'idMeal': '52768'},
  {'strMeal': 'Bakewell tart',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/wyrqqq1468233628.jpg',
   'idMeal': '52767'},
  {'strMeal': 'Banana Pancakes',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/sywswr1511383814.jpg',
   'idMeal': '52855'},
  {'strMeal': 'Battenberg Cake',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/ywwrsp1511720277.jpg',
   'idMeal': '52894'},
  {'strMeal': 'BeaverTails',
   'strMealThumb': 'https://www.themealdb.com/images/media/meals/ryppsv1511815505.jpg',
 

### Tool Use Exercise  — “Find meals that contain shrimp (but try more than one method)”

Goal:
We want to find meals that contain **shrimp** (or “prawn”) — BUT TheMealDB does not have a direct “ingredient search” endpoint.  
So the LLM must propose ways to *approximate* this.

We will ask the LLM for NOT just one approach — but at least **two different** possible tool-use strategies.

**Prompt to LLM:**

I want to find meals that contain shrimp. TheMealDB does not have a direct `ingredient` search endpoint.  
Give me **two different** possible approaches I could take using TheMealDB endpoints to identify shrimp meals.

Rules:
- You must reference actual endpoints that exist (like `/search.php`, `/filter.php`, `/lookup.php`).
- For each approach, tell me which endpoint I should call FIRST, and what the parameter should be.
- Do NOT guess any meal names.  
- Do NOT claim you know which meals contain shrimp.  
- Only propose the *strategy*.


**After the LLM answers:**

→  Choose one approach  
→  Write the code to make that first API call  
→  Paste the resulting JSON **back into the LLM** and ask:

"Does this result still support the strategy? What do I do next?"

**Optional follow-up prompt ( ask LLM):**

This is the only approach you gave.  
Can you propose a *different* approach using a different endpoint flow?  
Again — do not tell me the answer directly — only propose the tool path.

→ Pick the second approach and test it as well.

---

**Note:**
The learning goal here is:  
The LLM is not allowed to “know the shrimp meals”.  
It is only allowed to propose *plans* to discover them using the tools (API calls).


In [12]:
## METHOD 1
## CODE:

import requests
import pandas as pd

# search by name for "shrimp" and "prawn"
def search_by_name():
    
    # shrimp
    url = "https://www.themealdb.com/api/json/v1/1/search.php"
    params_shrimp = {"s": "shrimp"}
    
    request_shrimp = requests.get(url, params=params_shrimp)
    shrimp_results = request_shrimp.json() # parse JSON response
    
    # prawn
    params_prawn = {"s": "prawn"}
    request_prawn = requests.get(url, params=params_prawn)
    prawn_results = request_prawn.json() # parse JSON response
    
    # combine results
    meals = [] # making list to populate with meals 
    
    if shrimp_results.get('meals'):
        meals.extend(shrimp_results['meals']) # adding shrimp result meals to meal list
    
    if prawn_results.get('meals'):
        meals.extend(prawn_results['meals']) # adding prawn result meals to the meal list 
    
    print(f"Found {len(meals)} meals with shrimp/prawn in name")
    
    # results
    if meals:
        for meal in meals: # if meals list exists, cycle through and print each one and its ID #
            print(f"{meal['strMeal']} (ID: {meal['idMeal']})")
    
    return meals


method1_results = search_by_name()

Found 7 meals with shrimp/prawn in name
Shrimp Chow Fun (ID: 52953)
Kung Po Prawns (ID: 52946)
Salmon Prawn Risotto (ID: 52823)
Chilli prawn linguine (ID: 52839)
Prawn & Fennel Bisque (ID: 52922)
Laksa King Prawn Noodles (ID: 52821)
Spring onion and prawn empanadas (ID: 53040)


In [13]:
## METHOD 2
## CODE:


def filter_seafood():
    print("Getting seafood meals and checking ingredients")
    
    # meals from seafood category
    url_filter = "https://www.themealdb.com/api/json/v1/1/filter.php"
    params_seafood = {"c": "Seafood"}
    
    request_seafood = requests.get(url_filter, params=params_seafood)
    seafood_results = request_seafood.json() # parse JSON response
    
    if not seafood_results.get('meals'): # make sure that the meals being reccomended are actually seafood 
        print("No seafood meals found")
        return []
    
    print(f"Found {len(seafood_results['meals'])} seafood meals")
    
    # full details for each seafood meal and check ingredients (need to use i)
    url_lookup = "https://www.themealdb.com/api/json/v1/1/lookup.php"
    shrimp_meals = [] # empty list to populate with shrimp meals that I'm looking for
    
    for meal in seafood_results['meals'][:10]:  # first 10 to avoid too many API calls
        params_lookup = {"i": meal['idMeal']} # full details lookup by ID
        request_details = requests.get(url_lookup, params=params_lookup)
        meal_details = request_details.json() # parse JSON response
        
        if meal_details.get('meals'): # check if meals exist in response and assign it to full meal variable 
            full_meal = meal_details['meals'][0]
            
            # check all ingredient fields for shrimp/prawn
            ingredients_text = "" # empty string for ingredients population later on
            for i in range(1, 21):  # strIngredient1 through strIngredient20
                ingredient = full_meal.get(f'strIngredient{i}', '') # get ingredient for each element i unless its not found, then print empty string
                measure = full_meal.get(f'strMeasure{i}', '') # get measurements for that ingredient for each element i unless its not found, then print empty string
                if ingredient:
                    ingredients_text += f"{ingredient} {measure}, " # if ingredient exists, add the details to the text
            
            # check if shrimp or prawn appears in ingredients
            if 'shrimp' in ingredients_text.lower() or 'prawn' in ingredients_text.lower(): # name, ID, and ingredients preview for meals with shrimp/prawn in them
                shrimp_meals.append({
                    'name': full_meal['strMeal'],
                    'id': full_meal['idMeal'],
                    'ingredients_preview': ingredients_text[:100] + "..."
                })
    
    print(f"Found {len(shrimp_meals)} seafood meals containing shrimp/prawn:")
    for meal in shrimp_meals:
        print(f"- {meal['name']} (ID: {meal['id']})")
        print(f"  Ingredients: {meal['ingredients_preview']}")
    
    return shrimp_meals


method2_results = filter_seafood()

Getting seafood meals and checking ingredients
Found 31 seafood meals
Found 3 seafood meals containing shrimp/prawn:
- Fish pie (ID: 52802)
  Ingredients: Floury Potatoes 900g, Olive Oil 2 tbsp, Semi-skimmed Milk 600ml, White Fish Fillets 800g, Plain flou...
- Fish Stew with Rouille (ID: 52918)
  Ingredients: Prawns 6 large, Olive Oil 3 tbs, Dry White Wine 150ml, Fish Stock 200ml, Fennel 1 small finely diced...
- Garides Saganaki (ID: 52764)
  Ingredients: Raw king prawns 500g, Olive oil 3 tablespoons, Chopped onion 1, Freshly chopped parsley pinch, White...


## Section 2: How to Use an LLM respectfully

The LLM is **not** here to replace your thinking.

The LLM is here to:
- suggest *ideas*
- help you *communicate patterns*
- help you *frame claims that can be verified*

The LLM is **not allowed** to invent facts, ingredients, or data that the API did not provide.

### GOLDEN RULE

**LLM = language reasoning + framing**  
**API = actual facts / evidence**

We ALWAYS verify LLM claims against real data.

---

### Good LLM use examples

| Good (allowed) | Why |
|---|---|
| “Here are 2 ways to describe the difference between these meals.” | helping with communication |
| “Can you generate 2 testable claims that I can check in pandas?” | helping generate hypotheses |
| “Summarize the evidence table I pasted.” | summarizing what already exists |

### Not allowed

| Not allowed | Why |
|---|---|
| “Tell me the ingredients for this meal.” | API already gives this — model might hallucinate |
| “Decide which meal is safe without seeing the data.” | decision must be data-driven |
| “Guess which meals have peanuts.” | guessing = not acceptable |

---

### The pattern you must follow

1) **API → get real data**
2) **YOU → tidy & prepare it**
3) **YOU → paste only the relevant subset into the LLM**
4) **LLM → proposes claims (not answers)**
5) **YOU → verify each claim in code**

This is the professional workflow in data work:
> models generate possible patterns → humans verify with real data.

---

### If the LLM says something, that is just a *claim*.  
It is not true until we verify it.

We trust the API values — not the LLM wording.


## Using the endpoints with your own question + your own purpose

For section 2, you are not just asking the LLM to give you answers  
You are using proper LLM prompts and API calls as *tools* to support a real question that YOU care about.

Examples (you don’t have to pick these):
- meals you might want to cook this week(ingredients, regions, etc.)
- meals that match your personal taste (sweet, spicy, seafood, regions, etc.)
- meals that fit a budget or fit your macros (low ingredient count, high protein)
- meals that avoid something (allergy, dislike, dietary restriction)

The point is: **you decide what makes a meal “good” or “interesting.”**  
Then you use the LLM prompts ans API calls as tools to explore that idea.

---

### The 3 endpoints

| What you want to do | Endpoint | Parameter |
|---|---|---|
| search by name text | `/search.php` | `s` |
| filter by category | `/filter.php` | `c` |
| lookup full detail for a single meal | `/lookup.php` | `i` |

---

There are also some other endpoints that TheMealDB API provides, you may wanna check them out and see if they can be helpful for solving your question.
- See here: https://www.themealdb.com/api.php

### What we are practicing today

1) **Ask the LLM** what API call(s) would help answer your question  
   → the LLM suggests a *plan*, not the answer

2) **YOU run the API call(s)** in Python  
   → get the real data

3) **Ask the LLM again** what patterns might be interesting  
   → suggestions, not answers

4) **YOU verify** any claims with real code  
   → API data is the ground truth

**In this lab — The LLM helps with planning, not with answering.**



## What LLM Assistance *Is* Allowed for Code

You **may** ask the LLM to help you write code for:
- filtering ingredients based on your allergy / dislike tokens
- counting ingredients
- grouping meals by a flag (safe / not-safe)
- formatting results (DataFrame, dict, list, etc.)

Example of *allowed* ask:

> “Here is my ingredients DataFrame. How do I filter rows where `ingredient_norm` contains the token `peanut`?”


### But important: the LLM is ONLY allowed to write code based on the data YOU already pasted in the chat, bacause otherwise it may come up keys and parameters that don't exit, and the codes might not work.

---

## What is NOT allowed

- asking the LLM to invent ingredients
- asking the LLM to decide which meals are safe (without code)
- asking the LLM to tell you the answer directly
- letting the LLM write code that assumes columns that don’t exist

Example of **not allowed** ask:

> “Which meals contain peanuts? Just tell me.”  
❌ → That skips the verification/data responsibility step.

---

The *results* must always be produced by code that runs on the real API data — not by the model’s text.


In [ ]:
## CODE:

import requests
import pandas as pd


# find meals with few ingredients (low count) 
def find_quick_meals(max_ingredients=5):
    print(f"\nFinding meals with {max_ingredients} or fewer ingredients")
    
    # meals from simple categories
    categories = ['Breakfast', 'Starter', 'Side']
    meals = [] # meals list to populate later
    
    for category in categories:
        url_filter = "https://www.themealdb.com/api/json/v1/1/filter.php"
        params = {"c": category}
        request = requests.get(url_filter, params=params)
        results = request.json() # parse JSON response
        
        if results.get('meals'): # if response result meal exists
            # full details for each meal
            for meal in results['meals'][:5]:  # first 5 from each category
                url_lookup = "https://www.themealdb.com/api/json/v1/1/lookup.php"
                params_lookup = {"i": meal['idMeal']} # full detail lookup by ID
                request_details = requests.get(url_lookup, params=params_lookup) 
                meal_details = request_details.json() # parse JSON into dict
                
                if meal_details.get('meals'): # if meal details exist add it in first element of meals data/list
                    meals.append(meal_details['meals'][0])
    
    # ingredient count for each meal
    quick_meals = [] # quick meal list for later (populate)
    
    for meal in meals: # initialization for each meal to change later
        ingredient_count = 0 # count to change
        ingredients_list = [] # list to populate
        
        for i in range(1, 21): # get ingredient elements 1-20
            ingredient = meal.get(f'strIngredient{i}', '')
        
            if ingredient is not None: # if ingredient exists showcase it with no whitesapce
                ingredient = str(ingredient).strip()
                if ingredient and ingredient.lower() != 'null' and ingredient.lower() != '': # if ingredient isn't null/empty add it to the count and list of ingredients
                    ingredient_count += 1
                    ingredients_list.append(ingredient)
        
        if 0 < ingredient_count <= max_ingredients: # as long as the max number of ingredients is above 0 and within specified range, show the full meal details 
            quick_meals.append({
                'name': meal['strMeal'],
                'id': meal['idMeal'],
                'ingredient_count': ingredient_count,
                'category': meal.get('strCategory', 'Unknown'), # get meal category if it exists, else say its unknown
                'ingredients': ingredients_list
            })
    
    # sort by ingredient count (fewest first)
    quick_meals.sort(key=lambda x: x['ingredient_count'])
    
    print(f"Found {len(quick_meals)} meals with {max_ingredients} or fewer ingredients:")
    for meal in quick_meals:
        print(f"- {meal['name']} ({meal['category']})")
        print(f"  Ingredients: {meal['ingredient_count']} total - {', '.join(meal['ingredients'])}") # meal ingredients separated/joined by commas
    
    return quick_meals


# meals that don't contain specific allergens
def filter_allergen_free(allergens=['nuts', 'peanuts', 'shellfish']):
    print(f"Finding meals without {allergens}")
    
    # sample meals from different categories
    categories = ['Chicken', 'Beef', 'Vegetarian']
    safe_meals = [] # list of safe meals to populate later based on allergens in these categories
    
    for category in categories:
        url_filter = "https://www.themealdb.com/api/json/v1/1/filter.php"
        params = {"c": category}
        request = requests.get(url_filter, params=params)
        results = request.json() # parse JSON
        
        if results.get('meals'): # if meals exist get full details and ID for first 3 meals in each category
            # first 3 meals from each category
            for meal in results['meals'][:3]:
                url_lookup = "https://www.themealdb.com/api/json/v1/1/lookup.php"
                params_lookup = {"i": meal['idMeal']}
                request_details = requests.get(url_lookup, params=params_lookup)
                meal_details = request_details.json() # parse JSON into dict
                
                if meal_details.get('meals'): # if meal details exist add it in first element 
                    full_meal = meal_details['meals'][0]
                    
                    # check if meal contains any allergens
                    contains_allergen = False # set to change later if needed
                    all_ingredients = [] # ingredient list to populate 
                    
                    for i in range(1, 21): # ingredients 1-20
                        ingredient = full_meal.get(f'strIngredient{i}', '') # get ingredient if it exists, else empty string
                        if ingredient is not None:
                            ingredient = str(ingredient).lower().strip() # format ingredient if it exists
                            if ingredient and ingredient != 'null' and ingredient != '': # if it exists and is not null/empty append it
                                all_ingredients.append(ingredient)
                                # check if ingredient matches any allergen
                                for allergen in allergens:
                                    if allergen in ingredient:
                                        contains_allergen = True
                                        break
                    
                    if not contains_allergen: # if no allergens in ingredients, add to list of safe meals with full meal details 
                        safe_meals.append({
                            'name': full_meal['strMeal'],
                            'id': full_meal['idMeal'],
                            'category': full_meal.get('strCategory', 'Unknown'), # get category if it exists, else show as unknown
                            'ingredients': all_ingredients
                        })
    
    print(f"Found {len(safe_meals)} meals without the specified allergens:")
    for meal in safe_meals:
        print(f"- {meal['name']} ({meal['category']})")
        print(f"  Safe ingredients: {', '.join(meal['ingredients'][:4])}{'...' if len(meal['ingredients']) > 4 else ''}") # join ingredients with comma, and if more than 4 ingredient in total, add ... to string's end, else leave empty
    
    return safe_meals


allergen_free_results = filter_allergen_free(['nuts', 'peanut']) # can change allergens 




quick_meals_results = find_quick_meals(max_ingredients=6) # can change ingredient count 


Finding meals without ['nuts', 'peanut']
Found 9 meals without the specified allergens:
- 15-minute chicken & halloumi burgers (Chicken)
  Safe ingredients: chicken breasts, oil, hotsauce, lemon juice...
- Ayam Percik (Chicken)
  Safe ingredients: chicken thighs, challots, ginger, garlic clove...
- Brown Stew Chicken (Chicken)
  Safe ingredients: chicken, tomato, onions, garlic clove...
- Aussie Burgers (Beef)
  Safe ingredients: lean minced steak, cooked beetroot, naan bread, rocket...
- Beef and Mustard Pie (Beef)
  Safe ingredients: beef, plain flour, rapeseed oil, red wine...
- Beef and Oyster pie (Beef)
  Safe ingredients: beef, olive oil, shallots, garlic...
- Baingan Bharta (Vegetarian)
  Safe ingredients: aubergine, onion, tomatoes, garlic...
- Beetroot Soup (Borscht) (Vegetarian)
  Safe ingredients: beetroot, olive oil, chicken stock cube, water...
- Cabbage Soup (Shchi) (Vegetarian)
  Safe ingredients: unsalted butter, onion, cabbage, carrots...

Finding meals with 6 or fewer